In [ ]:
import math
import statistics
import numpy as np
import seaborn as sns
import pandas as pd
import hydrostats as hs
import cartopy.crs as ccrs
from fastdtw import fastdtw
import matplotlib.pyplot as plt
from sklearn import linear_model
from scipy.stats import gaussian_kde
from datetime import datetime
from netCDF4 import Dataset, num2date
from sklearn.metrics import r2_score, mean_squared_error
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
#data_year = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
data_year = ['2014','2015','2016','2017','2018','2019','2020']

for i in data_year:    
    a_hour = []
    a_day = []
    a_month = []
    a_year = []
    a_val = []

    satfile = Dataset(rf'C:\Users\ydjoe\Documents\solar radiation\nc_surface_solar_radiation_downwards\Dublin_surface_solar_radiation_downwards_' + i + '.nc', 'r')
    all_t2m = satfile.variables['ssrd'][:]
    nctime = satfile.variables['time'][:] # Isolate all time values
    t_unit = satfile.variables['time'].units # Isolate time units
    t_cal = satfile.variables['time'].calendar # Isolate time calendar
    satfile.close() # Close file
    satdate = [] # Initialise satellite dates list
    satdate.append(num2date(nctime, units = t_unit, calendar = t_cal))

    for k in range(len(all_t2m)):
        a_hour.append(int(satdate[0][k].strftime("%H")))
        a_day.append(int(satdate[0][k].strftime("%d")))
        a_month.append(int(satdate[0][k].strftime("%m")))
        a_year.append(int(satdate[0][k].strftime("%Y")))
        a_val.append(round(all_t2m[k,:,:][2][2]/(3600)))

    sat_df = pd.DataFrame(list(zip(a_year,a_month,a_day,a_hour,a_val)),columns=['Year','Month','Day','Hour','Val'])
    sat_df.to_csv(rf'C:\Users\ydjoe\Documents\solar radiation\surface_solar_radiation_downwards_2\Dublin_sat_solar_radiation_{i}.csv')

## boxplots

In [ ]:
months_lst = [
'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',  'December'
]

mth2 = []
for i in months_lst:
    mth2.append(i[:3])

pos_ls = []
for i in range(12):
    pos_ls.append(i+1)

In [ ]:
#data_year = ['2014','2015','2016','2017','2018','2019','2020']
data_year = ['2020']

font = {'size'   : 12}

plt.rc('font', **font)

#fig, axes = plt.subplots(        nrows=7, ncols=1, figsize=(7,20), dpi=80, facecolor="w", edgecolor="k",sharex=False    )
#iter = 0
for i in data_year:  
    sat_df = pd.read_csv(rf'C:\Users\ydjoe\Documents\solar radiation\surface_solar_radiation_downwards/Dublin_sat_solar_radiation_{i}.csv',
                                  index_col = 0)
    df_land = pd.read_csv(rf'C:\Users\ydjoe\Documents\solar radiation\land_data/Dublin_land_solar_radiation_{i}.csv',
                                  index_col = 0)
    sat_solar_rad = sat_df['Val'].to_list()
    land_solar_rad = df_land['[W/m2]'].to_list()

    solar_diff = [a - b for a, b in zip(land_solar_rad, sat_solar_rad)]

    new_df = sat_df.copy()
    new_df['solar_diff'] = solar_diff
    new_df = new_df.drop(new_df.columns[4],axis=1)

    solar_diff_hour_df = new_df[new_df.Hour==12]
    solar_diff_hour_df = solar_diff_hour_df.drop(solar_diff_hour_df.columns[3],axis=1)
    solar_diff_hour_df.reset_index(drop=True, inplace=True)

    solar_diff_mean_lst = new_df.groupby(['Month','Day'])['solar_diff'].mean().to_list()
    solar_diff_hour_df['solar_diff'] = solar_diff_mean_lst

    month_unq = solar_diff_hour_df.Month.unique()
    plt_arr = []
    for j in month_unq:
        plt_arr.append(solar_diff_hour_df[solar_diff_hour_df.Month==j].solar_diff.values)
    #plt.sca(axes[iter]) 
    title = '2020 - Dublin Mean Satellite and Land Radiation Difference'
    #plt.title(title)
    plt.xlabel('Months')
    plt.ylabel('Differential GHI (W/m2)')
    plt.boxplot(plt_arr)
    plt.xticks(pos_ls,mth2)
    plt.tight_layout()

    iter+=1
plt.savefig('2020 - Dublin Mean Satellite and Land Radiation Difference.pdf', bbox_inches = 'tight', pad_inches = 0.05)

## linear regression

In [ ]:
data_year = ['2014','2015','2016','2017','2018','2019','2020']
ls_df_years = []
ls_df_sat = []
ls_df_land = []

for i in data_year:  
    sat_df = pd.read_csv(rf'C:\Users\ydjoe\Documents\solar radiation\surface_solar_radiation_downwards/Dublin_sat_solar_radiation_{i}.csv',
                                  index_col = 0)
    df_land = pd.read_csv(rf'C:\Users\ydjoe\Documents\solar radiation\land_data/Dublin_land_solar_radiation_{i}.csv',
                                  index_col = 0)
    df_land = df_land.rename(columns={'[W/m2]': 'Val'})
    
    ls_df_land.append(df_land)
    ls_df_sat.append(sat_df)
    
df_land_full = pd.concat(ls_df_land)
df_sat_full = pd.concat(ls_df_sat)

print(df_sat_full.shape)   
print(df_land_full.shape)   
#print(df_sat_full.head(24))

df_sat_full.Val = df_sat_full.Val.shift(-3)
df_sat_full['Val'] = df_sat_full['Val'].fillna(0)

df_sat_full['Month'] = df_sat_full['Month'].map(str)
df_sat_full['Day'] = df_sat_full['Day'].map(str)
df_sat_full['Hour'] = df_sat_full['Hour'].map(str)

df_sat_full.drop(df_sat_full.columns[[0]], axis=1, inplace=True) 
print(df_sat_full.shape)   
print(df_sat_full.head())

## lin reg

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


reg_1_coeff = []

train_r2_scores = []

test_r2_scores = []


i = 1
iter = i-1    
X = df_sat_full[df_sat_full.Month==str(i)]
X = pd.get_dummies(data=X)
X['target'] = df_land_full[df_land_full.Month==i].Val.values
y_for_plot = df_land_full[df_land_full.Month==i].Val.values


X_test = X

X_test= X_test[
    (X_test['Hour_10'] == (1)  ) | 
    (X_test['Hour_11'] == (1)  ) | 
    (X_test['Hour_12'] == (1)  ) | 
    (X_test['Hour_13'] == (1)  ) 
   ]


# X_test= X_test[
#     (X_test['Hour_9'] == (1)  )  |
#     (X_test['Hour_10'] == (1)  ) | 
#     (X_test['Hour_11'] == (1)  ) | 
#     (X_test['Hour_12'] == (1)  ) | 
#     (X_test['Hour_13'] == (1)  ) | 
#     (X_test['Hour_14'] == (1)  ) | 
#     (X_test['Hour_15'] == (1)  )     ]

#idx_pos = X_test.index


predictions = X_test['Val']
y_test = X_test['target']

X_test.drop('target',axis=1,inplace=True)
#print(y_test,X_test.head())
#df_sat_full.reset_index(drop=True, inplace=True)


reg1 = LinearRegression().fit(X_test,y_test)

# train_score = reg1.score(X_train,y_train)
# test_score  = reg1.score(X_test, y_test)

# train_r2_scores.append(train_score)
# test_r2_scores.append(test_score)

# reg_1_coeff.append(reg1.coef_) 

#plt.figure()    
# print(len(y_test),len(X_test))
# print(len(y_test),X_test.shape,X_train.shape)
# print(X_test.columns)

#plt.sca(axes[iter // 2, iter % 2])     
#plt.title(months_lst[iter])


#y_test = x_for_plot
#predictions = y_pred
print(len(predictions),len(y_test))
xy = np.vstack([y_test,predictions])
z = gaussian_kde(xy)(xy)


plt.scatter(x = predictions,y = y_test,c=z, s = 80)
sns.regplot(x = predictions,y = y_test,scatter = False,ci = 90, fit_reg=True,color='purple')
sns.regplot(x = predictions,y = y_test,scatter = False,ci = 0, fit_reg=True,color='purple')
plt.xlabel('Satellite Data')   
plt.ylabel(' Ground-based Sensor Data') 
plt.tight_layout()

#sns.regplot(x = y_train,y = predictions,scatter_kws={"color": "red"}, line_kws={"color": "black"})
#plt.plot(predictions)

#acc = mean_squared_error(y_test, predictions,squared=False)
#mse.append(acc)

plt.savefig('Jan - all.pdf', bbox_inches = 'tight', pad_inches = 0.05)
#lst_reg_1_score
